### Exercise 5.1: Likelihood model optimization

This exercise uses the Python programming language. We will make use of the statistical libraries *scipy* and *numpy* to generate data under a parametric model (a model that takes one or more variables as inputs which affect its results) to learn how likelihood can help us to estimate the correct values of parameters given data generated under a model. 

In [1]:
import scipy.optimize as so
import numpy
import toyplot

### Optimization

Maximum likelihood optimization is a statistical method for finding the best fitting set of parameters for a model. A likelihood function can be made up of many independent likelihood functions, where each describes the result of a trial relative to a statistical distribution. Let's consider a simple example of a coin flip, and estimating whether the coin is fair (i.e., whether p=0.5).


### Likelihood equation

The likelihood is a statement about probability. For a coin toss, there are two possible outcomes: heads or tails. If we observe 10 coin tosses and learn that heads came up 5 times, and tails came up 5 times, we can try to learn from this what the probability is of each of the possible outcomes. Sounds like the probability is around 0.5, right? But what if we observe 38 heads and 46 tails, should we still think the true probability is around 0.5? Well, we can use likelihood to tell us. 

Since there are only two possible outcomes to a coin toss we know that the probability of flipping heads (p) plus the probability of flipping tails (q) must sum to 1 (p + q = 1). *This is called the joint probability distribution of our model*. Because the probability of flipping tails (q) is simply 1 - p, it is fully conditional on p, and so in reality there is only one parameter to this model. Our goal then, put simply, is to estimate the probability that one event will come up heads (p). 

### Calculating likelihoods

Let's say the true probability of flipping heads for a coin is 0.5. Without knowing this, we can try to test if this is true by calculating the likelihood of a series of coin flips that are performed using this coin. **We are observing data and trying to estimate a parameter of the model that is most likely to produce the results we observe**. 

### Example

Probability theory tells us that the probability of many *independent* events is the product of their individual probabilities. In other words, the probability of five coin flips in a row coming up heads is `p * p * p * p * p`. Let's look at two examples. If the coin is fair (p=0.5) then the probability of five heads in a row is quite low (0.03), but if the coin is totally rigged (p=0.99) then it is super likely that we will see five heads in a row (0.95). From these observations, we could probably make a guess as whether the coin toss is fair or biased, but the more tosses we do the more accurate our estimate will be.

In [2]:
# if the coin is fair (p=0.5) then the probability isn't very high
p = 0.5
p * p * p * p * p


0.03125

In [3]:
# but if the coin is really unfair then the probability if quite high
p = 0.99
p * p * p * p * p

0.9509900498999999

### Making things more concise
We can describe the same calculation above more concisely using exponents. So from now on we'll describe the product of n independent trials with probability p as $p^n$. 

In [4]:
# the probability of observing 20 heads for a coin with p=0.6
p = 0.6
n = 20
p**n

3.6561584400629733e-05

In [5]:
# the probability of observing 10 heads and 10 tails for p=0.6
p = 0.6
q = 1 - p
np = 10
nq = 10

p**np * q**nq

6.340338096537601e-07

### The goal of *Maximum* likelihood:
We want to find the best possible parameter to explain our observed data. This can be done in one of two ways: mathematically or empirically. Mathematically, if a likelihood equation is easy to solve then we can take the derivative of the equation and set it equal to zero and solve for our parameter. However, for many complex likelihood functions this is too difficult to solve. Therefore, a frequent approach is to maximize the likelihood empirically by trying many different values using a heuristic search. Computers are great for this. Since this is the practice in phylogenetics we will focus on heuristic optimization search.

In [6]:
# our observed data
np = 62
nq = 40

Here we print the parameter of value of p used to calculate the likelihood, and the likelihood score next to each other on each line. You can see that the value of 0.6 has the highest likelihood... but it's kind of hard to interpret because all of the likelihood values are such small numbers. 

In [7]:
# let's see which parameter for p best fits the data
for p in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    likelihood = p**np * (1-p)**nq
    print("p={}; likelihood={}".format(p, likelihood))

p=0.1; likelihood=1.478088294143466e-64
p=0.2; likelihood=6.12998216346359e-48
p=0.3; likelihood=2.4290664364642646e-39
p=0.4; likelihood=2.8429516759252963e-34
p=0.5; likelihood=1.9721522630525295e-31
p=0.6; likelihood=2.1270474435739034e-30
p=0.7; likelihood=3.026416316092381e-31
p=0.8; likelihood=1.0783978666860197e-34
p=0.9; likelihood=1.4555783429306787e-43


For this reason, people usually look at the negative log of the likelihood, which is easier to interpret. Although the method is called "maximum likelihood", when working with the negative log-likelihood we are actually trying to *minimize* this score, which still means finding the parameter that best fits the data. Below you can see that for p=0.6 the -loglik score is lowest (68.32). 

In [8]:
# let's see which parameter for p best fits the data
for p in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    likelihood = p**np * (1-p)**nq
    print("p={}; -loglik={:.2f}".format(p, -numpy.log(likelihood)))

p=0.1; -loglik=146.97
p=0.2; -loglik=108.71
p=0.3; -loglik=88.91
p=0.4; -loglik=77.24
p=0.5; -loglik=70.70
p=0.6; -loglik=68.32
p=0.7; -loglik=70.27
p=0.8; -loglik=78.21
p=0.9; -loglik=98.64


### Functions
A cleaner and easier way to calculate the log-likelihood for our equation when using computer programs is to write a function. This is simply a tool that we can reuse over and over again to perform the same task of computing a result given some input variables. 

In [9]:
def coin_flip_log(p, nheads, ntails):
    ## calculate likelihood
    logp = nheads*numpy.log(p) + ntails*numpy.log(1.-p)
    
    ## return negative log-likelihood
    return -1*logp

In [10]:
coin_flip_log(0.5, 100, 100)

138.62943611198907

### Exhaustive parameter search
There are much more advanced methods for finding exact optimized parameters than to simply plug in every possible value by hand. The function below is from the scipy library and is a maximum likelihood optimization algorithm. It proposes new parameters based on the scores of the ones it searches previously, and keeps going until it finds the best value. For a simple problem like this it is super fast. For much more complex problems these computations can become quite computationally intensive. 

In the example below I give run the maximum likelihood (ML) optimization for our likelihood function when the data (args) is 50 heads and 200 tails. The ML optimized parameter value of p is 0.2, which sounds correct, since 50/250 trials is 20%. 

In [11]:
# starting value=0.5; observed flips = (50, 200)
so.fmin(coin_flip_log, x0=(0.5), args=(50, 200), disp=0)[0]

0.19999999999999973

Here is another trial where we enter a different set of observations. Now when the data is 133 heads and 385 tails the ML parameter estimate of p is 0.2567. 

In [12]:
# starting value=0.5; observed flips = (133, 385)
so.fmin(coin_flip_log, x0=(0.5), args=(133, 385), disp=0)[0]

0.25673828124999976

### Plot the likelihood over different parameter inputs

The first plot shows the likelihood calculated at different values for p between 0 and 1 when the observed data is heads=50, tails=200. It finds the optimum value for p at around 0.2. The second plot shows the likelihood when the observed data is heads=50, tails=50, and the optimum looks very close to 0.5. 

In [13]:
## generate data across 100 equally spaced points for lambda
data = [coin_flip_log(p, 50, 200) for p in numpy.linspace(0.01, 0.99, 100)]
    
## plot the likelihood surface
toyplot.plot(
    b=numpy.log(data), 
    a=numpy.linspace(0.01, 0.99, 100),
    width=500, height=300,
    ylabel="-log-likelihood", 
    xlabel="probability of heads");

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t985f985f24f3412da3ec20ebd7359e03" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 300.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 54.0 192.9946309645687 L 57.95959595959596 206.83302382472723 L 61.91919191919192 215.607778945671 L 65.87878787878788 222.05983503169207 L 69.83838383838383 227.1227133196919 L 73.79797979797979 231.2376328614142 L 77.75757575757575 234.65175455946354 L 81.71717171717171 237.51914816285415 L 85.67676767676767 239.9432669156904 L 89.63636363636363 241.99751558934173 L 93.59595959595958 243.73625526635738 L 97.55555555555554 245.20114882288485 L 101.51515151515152 246.4250396482694 L 105.47474747474747 247.43443623289863 L 109.43434343434345 248.25116612084227 L 113.39393939393939 248.89351291031994 L 117.35353535353536 249.37701952808618 L 121.31313131313132 249.71506924146325 L 125.27272727272725 249.91931461680593 L 129.23232323232324 250.0 L 133.1919191919192 249.9662078931749 L 137.15151515151516 249.82604994530035 L 141.11111111111111 249.58681698093784 L 145.07070707070707 249.25509829509934 L 149.03030303030303 248.83687758627002 L 152.989898989899 248.33761091983155 L 156.94949494949495 247.76229071857068 L 160.9090909090909 247.11549877823722 L 164.8686868686869 246.4014505815003 L 168.8282828282828 245.62403265126287 L 172.78787878787878 244.78683428863727 L 176.74747474747474 243.89317474373405 L 180.7070707070707 242.94612664210106 L 184.66666666666666 241.94853631727057 L 188.62626262626264 240.903041566921 L 192.58585858585857 239.81208724683427 L 196.54545454545453 238.6779390359513 L 200.5050505050505 237.5026956421028 L 204.46464646464648 236.2882996674535 L 208.4242424242424 235.0365473123682 L 212.3838383838384 233.74909706401917 L 216.34343434343435 232.42747748988532 L 220.30303030303028 231.07309423499458 L 224.26262626262624 229.6872363043078 L 228.22222222222223 228.27108169721802 L 232.18181818181816 226.8257024491008 L 236.14141414141415 225.35206912469633 L 240.1010101010101 223.8510547994201 L 244.06060606060606 222.3234385571512 L 248.020202020202 220.76990852636828 L 251.97979797979798 219.19106447045667 L 255.93939393939394 217.58741994240367 L 259.8989898989899 215.95940400874954 L 263.85858585858585 214.3073625424246 L 267.8181818181818 212.63155907880602 L 271.77777777777777 210.9321752238321 L 275.73737373737373 209.20931059716193 L 279.6969696969697 207.46298228697555 L 283.6565656565656 205.69312378590615 L 287.6161616161616 203.899583369539 L 291.57575757575756 202.08212186965008 L 295.5353535353536 200.24040978356967 L 299.4949494949495 198.37402364835228 L 303.45454545454544 196.4824415933326 L 307.4141414141414 194.56503796653382 L 311.37373737373736 192.62107690850962 L 315.3333333333333 190.64970472054682 L 319.2929292929293 188.64994084147963 L 323.2525252525253 186.620667207006 L 327.21212121212125 184.5606157152095 L 331.17171717171715 182.46835345912498 L 335.1313131313131 180.34226530786222 L 339.09090909090907 178.18053331693125 L 343.0505050505051 175.98111231911312 L 347.010101010101 173.74170088005545 L 350.969696969697 171.45970658469972 L 354.9292929292929 169.13220433340504 L 358.88888888888886 166.75588594437897 L 362.8484848484848 164.32699884464586 L 366.8080808080808 161.84127093132312 L 370.7676767676768 159.29381771868032 L 374.72727272727275 156.67902653458952 L 378.6868686868687 153.99041060944813 L 382.6464646464646 151.22042312599393 L 386.60606060606057 148.3602172148257 L 390.5656565656566 145.39933174586457 L 394.5252525252525 142.32527333741862 L 398.4848484848485 139.12295013730943 L 402.44444444444446 135.7738

In [14]:
## generate data across 100 equally spaced points for lambda
data = [coin_flip_log(p, 50, 50) for p in numpy.linspace(0.01, 0.99, 100)]
    
## plot the likelihood surface
toyplot.plot(
    b=numpy.log(data), 
    a=numpy.linspace(0.01, 0.99, 100),
    width=500, height=300,
    ylabel="-log-likelihood", 
    xlabel="probability of heads");

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t9f2756761de74eb0800c42662393d781" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 300.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 54.0 50.0 L 57.95959595959596 75.59942462705776 L 61.91919191919192 92.5696598718824 L 65.87878787878788 105.66237823364624 L 69.83838383838383 116.4861364766279 L 73.79797979797979 125.79582767244452 L 77.75757575757575 134.0107374982615 L 81.71717171717171 141.38956294704923 L 85.67676767676767 148.10389428707254 L 89.63636363636363 154.27373295142758 L 93.59595959595958 159.986463719991 L 97.55555555555554 165.30777875560338 L 101.51515151515152 170.28834742864723 L 105.47474747474747 174.96808538854066 L 109.43434343434345 179.37899520312314 L 113.39393939393939 183.54711906281554 L 117.35353535353536 187.49391881451697 L 121.31313131313132 191.2372748472015 L 125.27272727272725 194.79222430378056 L 129.23232323232324 198.17151672691406 L 133.1919191919192 201.3860391363155 L 137.15151515151516 204.445145971582 L 141.11111111111111 207.35691855460968 L 145.07070707070707 210.1283715476392 L 149.03030303030303 212.7656190043293 L 152.989898989899 215.27400923340838 L 156.94949494949495 217.65823531605955 L 160.9090909090909 219.92242641764173 L 164.8686868686869 222.07022380124084 L 168.8282828282828 224.10484454476367 L 172.78787878787878 226.02913528995094 L 176.74747474747474 227.84561784558048 L 180.7070707070707 229.5565280827889 L 184.66666666666666 231.16384926575498 L 188.62626262626264 232.66934073300254 L 192.58585858585857 234.07456266670505 L 196.54545454545453 235.3808975474745 L 200.5050505050505 236.58956878125915 L 204.46464646464648 237.70165689647206 L 208.4242424242424 238.71811363833646 L 212.3838383838384 239.63977422984436 L 216.34343434343435 240.46736802179126 L 220.30303030303028 241.20152771582264 L 224.26262626262624 241.8427973125807 L 228.22222222222223 242.39163891048798 L 232.18181818181816 242.8484384583909 L 236.14141414141415 243.2135105463342 L 240.1010101010101 243.4871023024576 L 244.06060606060606 243.66939644982466 L 248.020202020202 243.7605135644487 L 251.97979797979798 243.7605135644487 L 255.93939393939394 243.66939644982466 L 259.8989898989899 243.4871023024576 L 263.85858585858585 243.21351054633433 L 267.8181818181818 242.8484384583909 L 271.77777777777777 242.39163891048787 L 275.73737373737373 241.8427973125807 L 279.6969696969697 241.20152771582264 L 283.6565656565656 240.46736802179126 L 287.6161616161616 239.63977422984436 L 291.57575757575756 238.71811363833646 L 295.5353535353536 237.70165689647206 L 299.4949494949495 236.5895687812593 L 303.45454545454544 235.3808975474745 L 307.4141414141414 234.07456266670505 L 311.37373737373736 232.66934073300254 L 315.3333333333333 231.16384926575498 L 319.2929292929293 229.5565280827889 L 323.2525252525253 227.84561784558048 L 327.21212121212125 226.02913528995094 L 331.17171717171715 224.10484454476367 L 335.1313131313131 222.07022380124084 L 339.09090909090907 219.92242641764173 L 343.0505050505051 217.65823531605955 L 347.010101010101 215.27400923340838 L 350.969696969697 212.7656190043293 L 354.9292929292929 210.1283715476392 L 358.88888888888886 207.35691855460968 L 362.8484848484848 204.445145971582 L 366.8080808080808 201.3860391363155 L 370.7676767676768 198.17151672691406 L 374.72727272727275 194.79222430378056 L 378.6868686868687 191.2372748472015 L 382.6464646464646 187.49391881451697 L 386.60606060606057 183.54711906281568 L 390.5656565656566 179.37899520312314 L 394.5252525252525 174.96808538854066 L 398.4848484848485 170.28834742864723 L 402.44444444444446 165.3077787556